In [ ]:
import re
import cv2
import numpy as np
import imageio
import os
import csv
import json
from PIL import Image
from io import BytesIO
import pandas as pd
import openai
import base64
import requests
import io

In [ ]:
# Define folder paths and file paths
input_file = '/content/dev_visual_description_raw.json'
output_file = '/content/clean_file.json'
final_output_file = '/content/dev_environment_description.json'

colom_to_remove = [
    ",,,,",
    ",,,",
    ",,",
    ", , , , ,",
    ", , , ,",
    ", , ,",
    ", ,",
    ",  ",
]

# Phrases to eliminate
phrases_to_remove = [
    "\n",
    ", mural.",
    "\n-",
    "Visible places:",
    "Identified places:",
    "buildings name, signs (signs on walls or windows), specific descriptions of the environment, traffic signs, bus stops, bike parking, or specific stores",
    "name of the place identified from navigation instruction",
    "Building names, advertisements, and specific stores or restaurants",
    "Buildings with signs for ",
    "specific descriptions of the environment",
    "There are no identifiable building names",
    "store/restaurant name",
    "and specific stores",
    "\u2019",
    "No specific names visible",
    "not clearly visible in the image",
    "sign not clearly visible in the image",
    "- Intersection type:",
    "Intersection type: ",
    "Intersections type",
    "Intersections type",
    "intersection type ",
    "intersections type ",
    "building names",
    "red brick building",
    "Red brick building",
    "intersection appears to be a",
    "name visible",
    "Signs on buildings",
    "Advertisements",
    "Advertisement",
    "advertisements",
    "advertisement",
    "visible",
    "Building names",
    "Market sign",
    "Market signs",
    "specific stores",
    "name not identifiable",
    "Intersection type",
    "intersection type",
    "not identifiable",
    "- Specific signs: ",
    "- Specific stores: ",
    "- Landmark name: ",
    "- Buildings:",
    "restaurant signs",
    "buildings name",
    "buildings names",
    "building name",
    "Building name",
    "sign on building",
    "signs on building",
    "on building windows",
    "on building window",
    "signs on walls or windows",
    "store/restaurant name",
    "signs on windows",
    "signs on walls",
    "sign on the wall",
    "sign on the walls",
    "sign on window",
    "on windows",
    "sign on the window",
    "sign on a building",
    "signs on buildings",
    "sign on building",
    "restaurant sign",
    "specific signs",
    "landmark name",
    "junction type",
    "traffic signs",
    "traffic sign",
    "Traffic signs",
    "Traffic sign",
    "traffic signals",
    "traffic signal",
    "street signs",
    "store sign",
    "crosswalk sign",
    "crosswalk signs",
    "Crosswalk sign",
    " store name",
    "No specific names\n \n \n \n",
    "descriptions of the environment",
    "but there's",
    "like ",
    "type ",
    "\u201d",
    "\u201c",
    "\u014d",
    "\u00d4",
    " name",
    " \n",
    "/",
    " .",
    ".",
    ";",
    "\ ",
    "\"",
    " or",
    " or ",
    "None",
    "none",
    "- Junction type: ",
    "- Junction type: ",
    "crosswalks",
    "crosswalk",
    ":",
    "on awnings",
    "on awning",
    "scaffoldings",
    "scaffolding",
    "Scaffoldings",
    "Scaffolding",
    "\u70e4\u8089\u5e97 ",
    "specific store",
    "on the right side",
    "on the left",
    "on the right",
    "(s)",
    "--",
    "with signs in another language",
    "in another language",
    "environment description",
    "buildings",
    "()",
    "There is not environment description",
    ", environment descriptions:",
    "- Specific stores restaurant:",
    "Specific",
    "\u2764\ufe0f ",
    ": \u70e4\u8089\u5e97",
    "with ",
    "- :",
    "(",
    ")",
    " s,",
    "on the windows",
    "outdoor dining",
    "B\u00d8",
    "including",
    "There is not environment description",

]

# Define a dictionary for all the replacement rules
replacements = {
    "bike parking": "Bike Rental",
    "Bank": "bank",
    "BANK": "bank",
    "Market": "MARKET",
    "market": "MARKET",
    "Dunkin'": "Dunkin",
    "stops": "stop",
    "stop": "Stop",
    "STOP sign": "STOP",
    "STOP": "Stop sign",
    "bus": "Bus",
    "Dunkin": "Dunkin' Donuts",
    "restaurants": "",
    "restaurant": "",
    "Restaurants": "",
    "Restaurant": "",
    "\u1edf": "o",
    "MARKET store": "MARKET",
    "Chase bank": "Chase",
    "Chase": "Chase bank",
    "TD bank": "TD",
    "Santander bank": "Santander",
    "bank Santander": "Santander",
    "Santander": "bank Santander",
    "TD": "TD bank",
    "intersection": "Intersection",
    "MARKET": "Market store",  # Replace all occurrences of MARKET
    "Intersection (4-way)": "4-way Intersection",
    "signs": "sign",
    "\u00e9": "e ",  # Accent character replacement
    "3-way Intersection": "T-Intersection",
    "VITAL store establishment": "Vital Climbing",
    "VITAL": "Vital Climbing",
    "CLIMBING": "Vital Climbing",
    "bicycle parking": "Bike Rental",
    "\u2764\ufe0f": "Love",
    "scaffolding,": "",  # Remove trailing commas
    ", and": ",",
    " ,": ",",
    ",": ", ",
    "  ": " ",
    "   ": " ",
}
# Load JSON content
with open(input_file, 'r') as file:
    data = json.load(file)

# Recursive function to clean text in a dictionary or list
def clean_text(obj):
    if isinstance(obj, str):
        # Remove lines that contain "I'm sorry" and replace phrases in phrases_to_remove with ""
        lines = obj.splitlines()
        filtered_lines = []
        for line in lines:
            if "I'm" not in line:  # Remove lines containing "I'm sorry"
                # If the line contains "Identified places:", remove everything after it
                if "Identified places:" in line:
                    line = line.split("Identified places:")[0].strip()  # Keep content before "Identified places:"
                # Replace each phrase in phrases_to_remove with an empty string
                for phrase in phrases_to_remove:
                    line = line.replace(phrase, "")
                for colom in colom_to_remove:
                    line = line.replace(colom, ",")
                # Replace "traffic light" with "intersection"
                # Define a dictionary for all the replacement rules
                for key, value in replacements.items():
                    line = line.replace(key, value)

                line = line.strip(',').strip()
                # Ensure that single commas alone are removed
                if line != ",":
                    filtered_lines.append(line)

        # Join back the remaining lines and return the cleaned string
        return "".join(filtered_lines).strip(',')
    elif isinstance(obj, list):
        return [clean_text(item) for item in obj if item not in (",", "")]
    elif isinstance(obj, dict):
        return {key.strip(): clean_text(value) for key, value in obj.items() if value not in ("", None, ",")}
    return obj

# Process the data
cleaned_data = clean_text(data)

# Save the initial cleaned data to a JSON file
with open(output_file, 'w') as file:
    json.dump(cleaned_data, file, indent=4)

print(f"Initial cleaned JSON data saved to: {output_file}")

# Reload the cleaned data from the JSON file
with open(output_file, 'r') as file:
    reloaded_data = json.load(file)

# Function to remove empty keys
def remove_empty_keys(obj):
    if isinstance(obj, dict):
        # Remove key-value pairs where value is empty string or None
        return {key: remove_empty_keys(value) for key, value in obj.items() if value not in ("", None, "signs", ", signs")}
    elif isinstance(obj, list):
        return [remove_empty_keys(item) for item in obj]
    return obj  # Return non-dictionary/list objects unchanged

# Remove keys with empty values
final_cleaned_data = remove_empty_keys(reloaded_data)

# Add "Starting point" to the first node's panoramic and "Target" to the last node's panoramic
def add_start_and_target(data):
    for instance in data:
        # Loop through the Node Description of each instance
        node_description = instance.get("Node Description", {})

        # Get the first and last node keys
        node_keys = list(node_description.keys())

        if node_keys:
            # Modify the first node's panoramic
            first_node_key = node_keys[0]
            node_description[first_node_key]["panoramic"] = ("Starting Point, " +
                node_description[first_node_key].get("panoramic", "")
            )

            # Modify the last node's panoramic
            last_node_key = node_keys[-1]
            node_description[last_node_key]["panoramic"] = ("Target, "+
                node_description[last_node_key].get("panoramic", "")
            )

    return data

# Apply the changes to the reloaded data
final_cleaned_data = add_start_and_target(final_cleaned_data)

# Save the final cleaned data to a new JSON file
with open(final_output_file, 'w') as file:
    json.dump(final_cleaned_data, file, indent=4)

print(f"Final cleaned JSON data with 'Starting point' and 'Target' added saved to: {final_output_file}")


Initial cleaned JSON data saved to: /content/clean_file.json
Final cleaned JSON data with 'Starting point' and 'Target' added saved to: /content/dev_environment_description.json


In [ ]:
from collections import OrderedDict
# Define folder paths and file paths
input_file = '/content/dev_environment_description.json'

# Load JSON content
with open(input_file, 'r') as file:
    data_ck = json.load(file)

# Function to combine keys into "description"
def combine_keys(data):
    for instance in data:
        node_description = instance.get("Node Description", {})
        for node_id, node_data in node_description.items():
            # Collect components for the description
            description_parts = []

            # Combine 'panoramic', 'Forward', and 'Backward' contents
            description_parts.append(node_data.get("panoramic", ""))
            description_parts.append(node_data.get("Forward", ""))
            description_parts.append(node_data.get("Backward", ""))

            # Create the "description" key
            description = ", ".join(filter(None, description_parts)).strip()

            # Create a new ordered dictionary with 'description' first
            ordered_node = OrderedDict()
            ordered_node["description"] = description

            # Add remaining keys ('On the left' and 'On the right') as they are
            for key in ["On the left", "On the right"]:
                if key in node_data:
                    ordered_node[key] = node_data[key]

            # Update the node with the ordered dictionary
            node_description[node_id] = ordered_node

    return data

# Apply the transformation
transformed_data = combine_keys(data_ck)

# Save the output to a file
output_file = "transformed_data.json"
with open(output_file, "w") as file:
    json.dump(transformed_data, file, indent=4)

print(f"Transformed data saved to: {output_file}")

Transformed data saved to: transformed_data.json


In [ ]:
import json

# Load the provided JSON file
file_path = "/content/transformed_data.json"  # Update with the correct file path

with open(file_path, "r") as file:
    data = json.load(file)

# Function to remove repeated places or landmarks in descriptions
def clean_repeated_places(node_desc):
    cleaned_desc = {}
    for node_id, details in node_desc.items():
        description_words = set(details["description"].split(", "))

        # Collect words from "On the left" and "On the right"
        left_words = set(details.get("On the left", "").split(", "))
        right_words = set(details.get("On the right", "").split(", "))
        #forward_words = set(details.get("Forward", "").split(", "))

        # Remove words that appear in either "On the left" or "On the right"
        cleaned_words = description_words - left_words - right_words# - forward_words

        # Update description
        cleaned_desc[node_id] = details.copy()
        cleaned_desc[node_id]["description"] = ", ".join(cleaned_words)

    return cleaned_desc

# Apply cleaning function to all instances
for instance in data:
    instance["Node Description"] = clean_repeated_places(instance["Node Description"])

# Save the cleaned data
cleaned_file_path = "/content/cleaned_transformed_data.json"
with open(cleaned_file_path, "w") as cleaned_file:
    json.dump(data, cleaned_file, indent=4)

print(f"Cleaned data saved to {cleaned_file_path}")


Cleaned data saved to /content/cleaned_transformed_data.json


In [ ]:
import json
from collections import OrderedDict
# Define folder paths and file paths
input_file = '/content/cleaned_transformed_data.json'

# Load JSON content
with open(input_file, 'r') as file:
    data_flat = json.load(file)

# Function to flatten and generate the actions list
def flatten_and_generate_actions(data):
    for instance in data:
        node_description = instance.get("Node Description", {})
        new_node_description = OrderedDict()  # To store flattened descriptions
        actions = []  # To store the sequence of actions
        counter = 1

        # Process each node and add its description
        for node_id, node_data in node_description.items():
            # Add the main description, check if it is empty or None
            description = node_data.get("description", "").strip()  # Strip to handle spaces
            if not description:
                description = "There is not environment description"
            new_node_description[str(counter)] = description
            actions.append("description")
            counter += 1

            # Add "On the left" if present
            if "On the left" in node_data:
                new_node_description[str(counter)] = node_data["On the left"]
                actions.append("On the left")
                counter += 1

            # Add "On the right" if present
            if "On the right" in node_data:
                new_node_description[str(counter)] = node_data["On the right"]
                actions.append("On the right")
                counter += 1

            # Add "On front" if present
            #if "Forward" in node_data:
             #   new_node_description[str(counter)] = node_data["Forward"]
              #  actions.append("In front")
               # counter += 1

        # Update the instance with the new flattened structure and actions
        instance["Node Description"] = new_node_description
        instance["actions"] = actions

    return data

# Apply the transformation
transformed_data = flatten_and_generate_actions(data_flat)

# Save the output to a file
output_file = "flattened_with_stop_actions.json"
with open(output_file, "w") as file:
    json.dump(transformed_data, file, indent=4)

print(f"Transformed data with 'Stop' action saved to: {output_file}")

Transformed data with 'Stop' action saved to: flattened_with_stop_actions.json


In [ ]:
import json

# Define folder paths and file paths
input_file = '/content/flattened_with_stop_actions.json'

# Load JSON content
with open(input_file, 'r') as file:
    data_open = json.load(file)

def convert_actions(data):
    # List to hold the transformed data
    transformed_data = []

    # Iterate over each instance in the data
    for instance in data:
        # Extract the Instance_id and Node Description
        instance_id = instance.get("Instance_id")
        node_description = instance.get("Node Description")
        actions = instance.get("actions", [])

        # Initialize variables for tracking actions
        converted_actions = []

        # Get all indices of "description" actions
        loc = [i + 1 for i, action in enumerate(actions) if action == "description"]

        # Generate links based on actions
        j = 0  # Track the current description node
        for index, action in enumerate(actions):
            if action == "description":
                if j < len(loc) - 1:
                    converted_actions.append({
                        "from": loc[j],
                        "to": loc[j + 1],
                        "action": "description"
                    })
                j += 1  # Move to the next description node
            elif action == "On the left":
                converted_actions.append({
                    "from": loc[j - 1],
                    "to": index + 1,
                    "action": "On the left"
                })
            elif action == "On the right":
                converted_actions.append({
                    "from": loc[j - 1],
                    "to": index + 1,
                    "action": "On the right"
                })
            #elif action == "In front":
             #   converted_actions.append({
              #      "from": loc[j - 1],
               #     "to": index + 1,
                #    "action": "In front"
                #})

        # Add the transformed instance to the result
        transformed_instance = {
            "Instance_id": instance_id,
            "Node Description": node_description,
            "actions": converted_actions
        }
        transformed_data.append(transformed_instance)

    return transformed_data

# Apply the transformation
transformed_data = convert_actions(data_open)

# Save the output to a file
output_file = "actions_file.json"
with open(output_file, "w") as file:
    json.dump(transformed_data, file, indent=4)

print(f"Transformed data with actions saved to: {output_file}")


Transformed data with actions saved to: actions_file.json


In [ ]:
import json

# Define file paths
input_data_file = '/content/actions_file.json'
directions_file = '/content/dev_visual_environment_directions.json'
output_file = '/content/updated_environment_descriptions.json'

# Load JSON data
with open(input_data_file, 'r') as file:
    action_data = json.load(file)

with open(directions_file, 'r') as file:
    direction_data = json.load(file)

# Convert directions data to a dictionary for faster access
directions_map = {item["Instance_id"]: item["Directions"] for item in direction_data}

# Update action descriptions with corresponding directions
for instance in action_data:
    instance_id = instance["Instance_id"]
    if instance_id in directions_map:
        directions = directions_map[instance_id]
        direction_index = 0  # Initialize direction index
        for action in instance["actions"]:
            if action["action"] == "description":
                if direction_index < len(directions):
                    action["action"] = directions[direction_index]
                    direction_index += 1

# Save the updated data
with open(output_file, 'w') as file:
    json.dump(action_data, file, indent=4)

print(f"Updated actions saved to: {output_file}")


Updated actions saved to: /content/updated_environment_descriptions.json


In [ ]:
import json

# Load the provided JSON file
file_path = "/content/updated_environment_descriptions.json"  # Update with the correct file path

with open(file_path, "r") as file:
    data = json.load(file)

# Function to remove duplicate places or landmarks in node descriptions
def clean_node_descriptions(node_desc):
    cleaned_desc = {}
    for key, desc in node_desc.items():
        unique_elements = list(dict.fromkeys(desc.split(",")))  # Remove duplicates while preserving order
        cleaned_desc[key] = ",".join(unique_elements)
    return cleaned_desc

# Apply cleaning function to all instances
for instance in data:
    instance["Node Description"] = clean_node_descriptions(instance["Node Description"])

# Save the cleaned data
cleaned_file_path = "/content/cleaned_environment_descriptions.json"
with open(cleaned_file_path, "w") as cleaned_file:
    json.dump(data, cleaned_file, indent=4)

print(f"Cleaned data saved to {cleaned_file_path}")

Cleaned data saved to /content/cleaned_environment_descriptions.json
